## Xception IMAGE CLASSICATION NETWORK

Import packages

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
import os, datetime
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

Assign training and validation paths to variables

In [ ]:
PATH = '/../../datasets/faces-small/'
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
train_real_dir = os.path.join(train_dir, 'real/')
train_fake_dir = os.path.join(train_dir, 'fake/')
validation_real_dir = os.path.join(validation_dir, 'real/')
validation_fake_dir = os.path.join(validation_dir, 'fake/')

Compute and verify size of validation and training sets

In [ ]:
num_real_tr = len(os.listdir(train_real_dir))
num_fake_tr = len(os.listdir(train_fake_dir))
num_real_val = len(os.listdir(validation_real_dir))
num_fake_val = len(os.listdir(validation_fake_dir))
total_train = num_real_tr + num_fake_tr
total_val = num_fake_val + num_fake_val

Set up variables for pre-processing dataset and training network

In [ ]:
batch_size = 64
epochs = 50
IMG_HEIGHT = 299
IMG_WIDTH = 299

Data formatter

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1. / 255,
                                           rotation_range=45,
                                           width_shift_range=.15,
                                           height_shift_range=.15,
                                           horizontal_flip=True,
                                           zoom_range=0.5)
validation_image_generator = ImageDataGenerator(rescale=1. / 255,
                                           rotation_range=45,
                                           width_shift_range=.15,
                                           height_shift_range=.15,
                                           horizontal_flip=True,
                                           zoom_range=0.5)

Load images and apply rescaling

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH))
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH))

Create models

In [ ]:
inputs = Input(shape=(IMG_HEIGHT, IMG_HEIGHT, 3))

# Create base Xception model
base_model = tf.keras.applications.Xception(include_top=False,
                                            input_tensor=Input(shape=(IMG_HEIGHT, IMG_HEIGHT, 3)))

# Freeze layers
for layer in base_model.layers[:116]:
    layer.trainable=False

x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output = tf.keras.layers.Dense(2, activation='softmax', name='Classification')(x)

# Create fine-tuned model
model = tf.keras.Model(inputs, output)

Compile model

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Load weights

In [ ]:
model.load_weights('../../checkpoints/Xception_base_small/')
# model.load_weights('./checkpoints/Xception_unfrozen_output_small/')

Define checkpoints

In [ ]:
checkpoint_dir = '../../checkpoints/Xception_unfrozen_output_small/'
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_dir,
                                                save_weights_only=True,
                                                save_best_only=True,
                                                monitor='val_accuracy',
                                                verbose=1)

Create TensorBoard callback

In [ ]:
log_dir = "../../log/Xception_unfrozen_output_small/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

Create EarlyStopping callback

In [ ]:
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Display model summary

In [ ]:
model.summary()

Train model using fit

In [ ]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    callbacks=[cp_callback, tensorboard_callback, es_callback],
    verbose=1
)

In [ ]:
model.save("../../saved_models/Xception_unfrozen_output_small/my_model.h5")